In [1]:
import os
from shapely.geometry import Point, Polygon, MultiPolygon
from datetime import datetime
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as ctx
import sqlite3 as sl
import itertools

import logging
logging.getLogger().setLevel(logging.INFO) # DEBUG, INFO or WARN

from dotenv import load_dotenv
load_dotenv()

from isochrones import Isochrones

DROOT = '../1-data/'
os.makedirs(os.path.join(DROOT, '3-interim', 'populationmasks'), exist_ok=True)

In [2]:
cities = pd.read_excel(os.path.join(DROOT, '1-research', 'cities.xlsx'), index_col=0)
cities.head(5)

,City,ctr_name,ctr_lat,ctr_lon,cells,Country,ID_HDC_G0
0,Vienna,Rathaus Wien,48.21080,16.35784,4381,AUT,3027
1,Graz,Hauptplatz Graz,47.06955,15.43383,1168,AUT,3000
2,Brussels,Brussels Town hall,50.84642,4.35232,3747,BEL,2135
3,Berlin,Old City Hall,52.51687,13.41160,5187,DEU,2851
4,Hamburg,Rathaus,53.55035,9.99232,4912,DEU,2531


In [3]:
done = ['Vienna', 'Graz', 'Brussels', 'Berlin', 'Hamburg', 'Copenhagen', 'Aarhus', 
        'Madrid', 'Barcelona', 'Paris', 'Marseille', 'London', 'Birmingham', 'York', 
        'Luxembourg', 'Amsterdam', 'Utrecht', 'Stockholm', 'Gothenburg', 'Helsinki', 
        'Oslo', 'Warsaw', 'Lisbon', 'Bern', 'Prague', 'Tallinn', 'Budapest', 'Dublin', 
        'Riga', 'Perth', 'Sydney', 'Melbourne', 'Adelaide', 'Auckland', 'Wellington', 
        'Mumbai', 'New Delhi', 'Kuala Lumpur', 'Toronto']

In [4]:
BING_KEY = os.environ['BING_API_KEY']
CACHE = os.path.join(DROOT, '3-interim', 'bing_cache.db')
client = Isochrones(bing_key=BING_KEY, db=CACHE)

In [5]:
for i, city in cities.iterrows():
    print(city.City)
    if city.City in done:
        continue;
        
    file = f'{city.ID_HDC_G0}.pcl'
    df = pd.read_pickle(os.path.join(DROOT, '3-interim', 'populationmasks', file))
    gdf = gpd.GeoDataFrame(df)
    
    origins  = enumerate(gdf.centroid.to_crs("EPSG:4326"))
    times    = [5, 15, 25, 35, 45]
    modes_dt = [
        ('driving', 'driving-peak', datetime(2023, 5, 2, 8, 30, 37)), 
        ('driving', 'driving-off', datetime(2023, 5, 2, 8, 30, 37)), 
        ('transit', 'transit-peak', datetime(2023, 5, 2, 8, 30, 37)),
        ('transit', 'transit',      datetime(2023, 5, 2, 13, 0, 37)), 
        ('walking', 'walking',      datetime(2023, 5, 2, 8, 30, 37))
    ]
    
    batch      = list(itertools.product(origins, times, modes_dt))
    isochrones = client.get_isochrones(city.ID_HDC_G0, batch)

Vienna
Graz
Brussels
Berlin
Hamburg
Copenhagen
Aarhus
Madrid
Barcelona
Paris
Marseille
London
Birmingham
York
Luxembourg
Amsterdam
Utrecht
Stockholm
Gothenburg
Helsinki
Oslo
Warsaw
Lisbon
Bern
Prague
Tallinn
Budapest
Dublin
Riga
Perth
Sydney
Melbourne
Adelaide
Auckland
Wellington
Mumbai
New Delhi
Kuala Lumpur
Toronto
Montreal


INFO:root:Out of total 5400, 100.00% cached.


Calgary


INFO:root:Out of total 2300, 100.00% cached.


New York


INFO:root:Out of total 26920, 83.78% cached.
INFO:root:Requesting 945-4510-driving-free-35m-v0
INFO:root:Requesting 945-4510-driving-free-45m-v0
INFO:root:Requesting 945-4511-driving-free-5m-v0
INFO:root:Requesting 945-4511-driving-free-15m-v0
INFO:root:Requesting 945-4511-driving-free-25m-v0
INFO:root:Requesting 945-4511-driving-free-35m-v0
INFO:root:Requesting 945-4511-driving-free-45m-v0
INFO:root:Requesting 945-4512-driving-free-5m-v0
INFO:root:Requesting 945-4512-driving-free-15m-v0
INFO:root:Requesting 945-4512-driving-free-25m-v0
INFO:root:Requesting 945-4512-driving-free-35m-v0
INFO:root:Requesting 945-4512-driving-free-45m-v0
INFO:root:Requesting 945-4513-driving-free-5m-v0
INFO:root:Requesting 945-4513-driving-free-15m-v0
INFO:root:Requesting 945-4513-driving-free-25m-v0
INFO:root:Requesting 945-4513-driving-free-35m-v0
INFO:root:Requesting 945-4513-driving-free-45m-v0
INFO:root:Requesting 945-4514-driving-free-5m-v0
INFO:root:Requesting 945-4514-driving-free-15m-v0
INFO:root

IndexError: list index out of range